# Calculating Historical Free Cash Flows

In this exercise we will load income statement and balance sheet data and use them to calculate free cash flows. 

## Load Financial Statements into `DataFrame`s

First we will use `pandas`' `read_excel` to get the data into `DataFrame`s.

In [1]:
import pandas as pd

all_statements_path = 'Exxon Mobil Corporation NYSE XOM Financials.xls'

### Income Statement

In [2]:

inc_df = pd.read_excel(all_statements_path, sheet_name='Income Statement')

In [3]:
inc_df.head()

,Unnamed: 0,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
0,,NaN,NaN,NaN,NaN,NaN,NaN
1,Revenue,364763,239854,200628,237162,279332,260812
2,Other Revenue,-,1552,-,-,-,-
3,Total Revenue,364763,241406,200628,237162,279332,260812
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see this is a little messy. We can clean this up first by setting the index to be the first column. You can pass an integer index for a column to say that that column should be used as the index of the `DataFrame`. Here we want this first column, so we will pass `index_col=0` into `read_excel`.

In [4]:
inc_df = pd.read_excel(all_statements_path, sheet_name='Income Statement', index_col=0)

In [5]:
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,-,1552,-,-,-,-
Total Revenue,364763,241406,200628,237162,279332,260812
NaN,NaN,NaN,NaN,NaN,NaN,NaN


That's starting to look better. But we can see that there are some empty rows in the data. We can also see that there is a `-` in some values where there should be missing data. We want to remove the rows without any data, but first we want to fill in missing values when there is `-`, so that we can make sure a row of completely `-` would get removed. The missing representation in `pandas` is `NaN`, which we can specify manually through `numpy.nan`.

In [6]:
import numpy as np

inc_df = inc_df.replace('-', np.nan)
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,NaN,1552,NaN,NaN,NaN,NaN
Total Revenue,364763,241406,200628,237162,279332,260812
NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we can see that those `-` got filled in for missing values. Now we want to remove the rows which have missing data. We can use `dropna` for this purpose. We must be careful to pass `how='all'` to `dropna` though. The default is `how='any'`, which means the row will be dropped if there is any missing value. Here we only want to drop the row if it is entirely missing, so we will use `how='all'`.

In [7]:
inc_df = inc_df.dropna(how='all')
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,NaN,1552,NaN,NaN,NaN,NaN
Total Revenue,364763,241406,200628,237162,279332,260812
Cost Of Goods Sold,234856,163605,132759,159053,190752,181228
Gross Profit,129907,77801,67869,78109,88580,79584


Now we can see that it has dropped the rows with all missing values, but not `Other Revenue` which has mostly missing values.

Now let's wrap this data cleaning process up into a function, because we're going to want to apply it to the balance sheet data as well.

In [8]:
def load_and_clean_statement_df(statements_path, sheet_name):
    df = pd.read_excel(statements_path, sheet_name=sheet_name, index_col=0)
    df = df.replace('-', np.nan)
    df = df.dropna(how='all')
    return df

inc_df = load_and_clean_statement_df(all_statements_path, 'Income Statement')
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,NaN,1552,NaN,NaN,NaN,NaN
Total Revenue,364763,241406,200628,237162,279332,260812
Cost Of Goods Sold,234856,163605,132759,159053,190752,181228
Gross Profit,129907,77801,67869,78109,88580,79584


Looks good. Let's use it on the balance sheet.

### Balance Sheet

Since we wrapped all this up in a function, now this process is very easy for the balance sheet.

In [9]:
bs_df = load_and_clean_statement_df(all_statements_path, 'Balance Sheet')
bs_df.head()

,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-09-30
Cash And Equivalents,4616,3705,3657,3177,3042,5351
Total Cash & ST Investments,4616,3705,3657,3177,3042,5351
Accounts Receivable,18541,13243,16033,21274,19638,25308
Other Receivables,9468,6632,5361,4323,5063,NaN
Total Receivables,28009,19875,21394,25597,24701,25308


## Working with the Data - `pandas` Method

Now we want to calculate free cash flow. As a reminder, here are the steps:
- Calculate non-cash expenses
- Calculate increase in working capital
- Calculate capital expenditures
- Calculate free cash flow from net income and the preceding items

### Calculate Non-Cash Expenses

For non-cash expenses, we need to total depreciation, amortization, stock-based compensation, impairment charges, and gains/losses on investments.

To see what we have in the two datasets, we can check the `.index` attribute of the `DataFrame`s.

In [10]:
inc_df.index

Index(['Revenue', 'Other Revenue', '  Total Revenue', 'Cost Of Goods Sold',
       '  Gross Profit', 'Selling General & Admin Exp.',
       'Exploration/Drilling Costs', 'Depreciation & Amort.',
       'Other Operating Expense/(Income)', '  Other Operating Exp., Total',
       '  Operating Income', 'Interest Expense', '  Net Interest Exp.',
       'Income/(Loss) from Affiliates', 'Currency Exchange Gains (Loss)',
       'Other Non-Operating Inc. (Exp.)', '  EBT Excl. Unusual Items',
       'Gain (Loss) On Sale Of Invest.', 'Gain (Loss) On Sale Of Assets',
       'Asset Writedown', '  EBT Incl. Unusual Items', 'Income Tax Expense',
       '  Earnings from Cont. Ops.', '  Net Income to Company',
       'Minority Int. in Earnings', '  Net Income',
       '  NI to Common Incl Extra Items', '  NI to Common Excl. Extra Items',
       'Basic EPS', 'Basic EPS Excl. Extra Items',
       'Weighted Avg. Basic Shares Out.', 'Diluted EPS',
       'Diluted EPS Excl. Extra Items', 'Weighted Avg. Dilu

Looking like we have depreciation, impairment (called asset writedown here) and gains/losses on investments and assets. There is no stock-based compensation so we will exclude that from the calculation.

To get the entire `Series` of data from the `DataFrame` by the `index` value, we can use `.loc` on the `DataFrame`.

In [11]:
inc_df.loc['Asset Writedown']

Dec-31-2014      NaN
Dec-31-2015      NaN
Dec-31-2016    -3600
Dec-31-2017    -2000
Dec-31-2018     -700
Sep-30-2019     -700
Name: Asset Writedown, dtype: object

That gives us each year's impairment. Notice that it's negative though, representing an expense. The calculation assumes it is positive. We can make it positive by using `abs` (absolute value). 

In [12]:
abs(inc_df.loc['Asset Writedown'])

Dec-31-2014     NaN
Dec-31-2015     NaN
Dec-31-2016    3600
Dec-31-2017    2000
Dec-31-2018     700
Sep-30-2019     700
Name: Asset Writedown, dtype: object

We can do math directly with `Series`.

In [13]:
inc_df.loc['Depreciation & Amort.']

Dec-31-2014    17297
Dec-31-2015    18048
Dec-31-2016    18708
Dec-31-2017    17893
Dec-31-2018    18045
Sep-30-2019    18403
Name: Depreciation & Amort., dtype: object

In [14]:
inc_df.loc['Depreciation & Amort.'] + abs(inc_df.loc['Asset Writedown'])

Dec-31-2014      NaN
Dec-31-2015      NaN
Dec-31-2016    22308
Dec-31-2017    19893
Dec-31-2018    18745
Sep-30-2019    19103
dtype: object

But what we notice is that if any value is missing, then it will cause the final calculation to be missing. This is not what we want. We want it to assume it is zero if it is missing. So let's fill the `DataFrame`s with zeroes if they are missing. We can do this using `fillna`.

In [15]:
inc_df = inc_df.fillna(0)
bs_df = bs_df.fillna(0)

In [16]:
abs(inc_df.loc['Asset Writedown'])

Dec-31-2014       0
Dec-31-2015       0
Dec-31-2016    3600
Dec-31-2017    2000
Dec-31-2018     700
Sep-30-2019     700
Name: Asset Writedown, dtype: object

In [17]:
inc_df.loc['Depreciation & Amort.'] + abs(inc_df.loc['Asset Writedown'])

Dec-31-2014    17297
Dec-31-2015    18048
Dec-31-2016    22308
Dec-31-2017    19893
Dec-31-2018    18745
Sep-30-2019    19103
dtype: object

Now we are getting the type of result we want. Let's go ahead and do the full calculation.

In [18]:
non_cash_expenses = (
    inc_df.loc['Depreciation & Amort.'] + 
    abs(inc_df.loc['Asset Writedown']) + 
    inc_df.loc['Gain (Loss) On Sale Of Invest.'] + 
    inc_df.loc['Gain (Loss) On Sale Of Assets']
)  # NOTE: split onto multiple lines for readability, it would function exactly the same on one line without parentheses

In [19]:
non_cash_expenses

Dec-31-2014    20443
Dec-31-2015    18232
Dec-31-2016    23990
Dec-31-2017    20227
Dec-31-2018    20738
Sep-30-2019    21096
dtype: object

## Calculate Increase in Working Capital

First we need to calculate the net working capial, then calculate the change in that. Here we need balance sheet data, so let's look at the `index` there.

In [20]:
bs_df.index

Index(['Cash And Equivalents', '  Total Cash & ST Investments',
       'Accounts Receivable', 'Other Receivables', '  Total Receivables',
       'Inventory', 'Deferred Tax Assets, Curr.', 'Restricted Cash',
       'Other Current Assets', '  Total Current Assets',
       'Gross Property, Plant & Equipment', 'Accumulated Depreciation',
       '  Net Property, Plant & Equipment', 'Long-term Investments',
       'Deferred Tax Assets, LT', 'Other Long-Term Assets', 'Total Assets',
       'Accounts Payable', 'Accrued Exp.', 'Short-term Borrowings',
       'Curr. Port. of LT Debt', 'Curr. Port. of Cap. Leases',
       'Curr. Income Taxes Payable', 'Other Current Liabilities',
       '  Total Current Liabilities', 'Long-Term Debt', 'Capital Leases',
       'Pension & Other Post-Retire. Benefits',
       'Def. Tax Liability, Non-Curr.', 'Other Non-Current Liabilities',
       'Total Liabilities', 'Common Stock', 'Retained Earnings',
       'Treasury Stock', 'Comprehensive Inc. and Other',
     

In [21]:
nwc = bs_df.loc['Accounts Receivable'] + bs_df.loc['Inventory'] - bs_df.loc['Accounts Payable']

In [22]:
nwc

2014-12-31     9933
2015-12-31    11414
2016-12-31    13312
2017-12-31    16565
2018-12-31    17533
2019-09-30     3562
dtype: object

To get a change, we can take advantage of `Series.shift`, which shifts all the values by a number of rows.

In [23]:
nwc.shift(1)

2014-12-31      NaN
2015-12-31     9933
2016-12-31    11414
2017-12-31    13312
2018-12-31    16565
2019-09-30    17533
dtype: object

We can see that with `shift(1)`, the value from last period is now in this period. So `nwc.shift(1)` is representing last year's net working capital. So then the change in net working capital is simply:

In [24]:
change_nwc = nwc - nwc.shift(1)

In [25]:
change_nwc

2014-12-31       NaN
2015-12-31      1481
2016-12-31      1898
2017-12-31      3253
2018-12-31       968
2019-09-30    -13971
dtype: object

### Calculate Capital Expenditures

Here we need to first get the change in net property, plant, and equipment, then add the current depreciation to get capital expenditures.

In [26]:
bs_df.loc['  Net Property, Plant & Equipment']

2014-12-31    252668
2015-12-31    251605
2016-12-31    244224
2017-12-31    252630
2018-12-31    247101
2019-09-30    257065
Name:   Net Property, Plant & Equipment, dtype: object

In [27]:
change_ppe = bs_df.loc['  Net Property, Plant & Equipment'] - bs_df.loc['  Net Property, Plant & Equipment'].shift(1)

In [28]:
change_ppe

2014-12-31      NaN
2015-12-31    -1063
2016-12-31    -7381
2017-12-31     8406
2018-12-31    -5529
2019-09-30     9964
Name:   Net Property, Plant & Equipment, dtype: object

In [29]:
capex = change_ppe + inc_df.loc['Depreciation & Amort.']

In [30]:
capex

2014-12-31      NaN
2015-12-31    16985
2016-12-31    11327
2017-12-31    26299
2018-12-31    12516
2019-09-30    28367
dtype: object

### Put it All Together into FCFs

In [31]:
fcf = inc_df.loc['  Net Income'] + non_cash_expenses - change_nwc - capex

In [32]:
fcf

Dec-31-2014      NaN
Dec-31-2015    15916
Dec-31-2016    18605
Dec-31-2017    10385
Dec-31-2018    28094
Sep-30-2019    21350
dtype: object

## Working with the Data - `finstmt` Method

There are some drawbacks to using plain `pandas` for these kinds of analyses:
- Different financial statements will have different names for items, so code will be tied to only one data provider
- Names are coming in with extra spaces, bad formatting
- From different data providers, some items are reported as negative numbers and others as positive, would need to handle this all for general code
- It's pretty inconvenient using `.loc` with the full row name.
- It's a little verbose to get a change in a variable
- For each variable, we need to think about which financial statement it comes from to be able to pull it, and then look up the appropriate name
- We had to remember to fill in zeroes or the results would not be correct
- Common calculations still need to be done manually (everyone calculates FCF, shouldn't it be easier?)

Because of these issues, I searched around for a solution to them, but could not find any. But the beauty of open source is anyone can develop a package and make it available to everyone. So I developed the `finstmt` package which handles all of these issues.

https://nickderobertis.github.io/py-finstmt/

### An Aside to Package Installation

To get started using it, we will need to install this package. It doesn't come with Anaconda because I created it! To install packages in Python, the general way is `pip install mypackage` replacing `mypackage` with whatever you want to install. You would run this command inside the `Anaconda Prompt`. An alternative is to run it through Jupyter. If you put `!` before a command in Jupyter, Jupyter interprets that as you wanting to run that on the command line and not in the Jupyter notebook. Let's see that here.

In [33]:
!pip install finstmt

You should consider upgrading via the 'pip install --upgrade pip' command.


You should see that type of output, and you should especially see `Successfully installed finstmt` to see that it worked. Now we are able to `import finstmt`. 

### Using `finstmt`

I have created three main classes (so far) to help working with the data. `IncomeStatements`, `BalanceSheets`, and `FinancialStatements`. Let's import them.

In [34]:
from finstmt import IncomeStatements, BalanceSheets, FinancialStatements

`finstmt` expects to receive your data with missing rows already removed, with data items as the index, and with dates as the columns. It is not necessary to fill in zeroes or take absolute values as those are handled by the package. The output of our `load_and_clean_statement_df` function in the beginning puts it in the perfect format for this. Let's reload the `DataFrame`s so that we haven't messed with them.

In [35]:
inc_df = load_and_clean_statement_df(all_statements_path, 'Income Statement')
bs_df = load_and_clean_statement_df(all_statements_path, 'Balance Sheet')

Now we want to create `IncomeStatements` from the income statements and `BalanceSheets` from the balance sheets. The way to do this is using `.from_df` methods of each.

In [36]:
inc_data = IncomeStatements.from_df(inc_df)

/home/nick/.local/share/virtualenvs/fin-model-course-eIFMSc8A/lib/python3.7/site-packages/finstmt/findata/database.py:77: UserWarning: Previously had revenue extracted from "Revenue". Replacing with value from "  Total Revenue"
  warnings.warn(f'Previously had {item_config.key} '


The first feature that we get from this package is much cleaner output out of the box. Simply display the data and it has cleaned up all the names of the variables, standardized the date format, and formatted values in currency format.

In [37]:
inc_data

,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,09/30/2019
Revenue,"$364,763","$241,406","$200,628","$237,162","$279,332","$260,812"
Cost of Goods Sold,"$234,856","$163,605","$132,759","$159,053","$190,752","$181,228"
Gross Profit,"$129,907","$77,801","$67,869","$78,109","$88,580","$79,584"
R&D Expense,-,-,-,-,-,-
SG&A Expense,"$12,002","$10,961","$11,783","$11,893","$12,300","$12,094"
Depreciation & Amortization Expense,"$17,297","$18,048","$18,708","$17,893","$18,045","$18,403"
Other Operating Expenses,"$64,857","$32,834","$31,375","$32,459","$35,230","$33,161"
Operating Expense,"$94,156","$61,843","$61,866","$62,245","$65,575","$63,658"
Earnings Before Interest and Taxes,"$34,082","$14,435","$4,536","$14,074","$21,539","$14,459"
Interest Expense,$286,$311,$453,$601,$766,$844


We can also see that it has made the impairment expense (called asset writedown before) into a positive number automatically.

The next main feature is easier access to variables. They have short names you can access via `.` and then the name. You can tab-complete all of these names. Go into the next cell, put your cursor after the `.`, and press tab. You might have to do it a couple times to get it to come up, but you will see all the variables.

In [ ]:
inc_data.

Now we can access those variables easily.

In [39]:
inc_data.revenue

2014-12-31    364763
2015-12-31    241406
2016-12-31    200628
2017-12-31    237162
2018-12-31    279332
2019-09-30    260812
dtype: int64

You can also see we get back to the raw numbers when accessing a variable, even though when we show the full statement, it is formatted.

We can also pull out one or more dates from the statements easily. Notice also that I'm not even using the same format of the dates, it understands that what you are passing is a date and converts it to match the column.

In [40]:
inc_data['2014-12-31']

,0
Revenue,"$364,763"
Cost of Goods Sold,"$234,856"
Gross Profit,"$129,907"
R&D Expense,-
SG&A Expense,"$12,002"
Depreciation & Amortization Expense,"$17,297"
Other Operating Expenses,"$64,857"
Operating Expense,"$94,156"
Earnings Before Interest and Taxes,"$34,082"
Interest Expense,$286


In [41]:
inc_data[['2014-12-31', '2015-12-31']]

,12/31/2014,12/31/2015
Revenue,"$364,763","$241,406"
Cost of Goods Sold,"$234,856","$163,605"
Gross Profit,"$129,907","$77,801"
R&D Expense,-,-
SG&A Expense,"$12,002","$10,961"
Depreciation & Amortization Expense,-,-
Other Operating Expenses,"$64,857","$32,834"
Operating Expense,"$94,156","$61,843"
Earnings Before Interest and Taxes,"$34,082","$14,435"
Interest Expense,$286,$311


Now let's go ahead and construct the balance sheet.

In [42]:
bs_data = BalanceSheets.from_df(bs_df)

In [43]:
bs_data

,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,09/30/2019
Cash and Cash Equivalents,"$4,616","$3,705","$3,657","$3,177","$3,042","$5,351"
Short-Term Investments,-,-,-,-,-,-
Cash and Short-Term Investments,"$4,616","$3,705","$3,657","$3,177","$3,042","$5,351"
Receivables,"$18,541","$13,243","$16,033","$21,274","$19,638","$25,308"
Inventory,"$16,678","$16,245","$15,080","$16,992","$18,958","$17,590"
"Deferred Tax Assets, Current","$2,001","$1,329",-,-,-,-
Other Current Assets,"$1,564","$1,469","$1,285","$1,368","$1,272","$1,759"
Total Current Assets,"$52,910","$42,623","$41,416","$47,134","$47,973","$50,008"
"Grosss Property, Plant & Equipment","$446,789","$447,337","$453,915","$477,185","$477,190",-
Accumulated Depreciation,"$194,121","$195,732","$209,691","$224,555","$230,089",-


We can work with the balance sheet in the same way as the income statement.

The package gets even more powerful when we combine the statements.

In [44]:
stmts = FinancialStatements(inc_data, bs_data)

We nice formatting of the statement, showing each nicely formatted statement with headers.

In [45]:
stmts

,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,09/30/2019
Revenue,"$364,763","$241,406","$200,628","$237,162","$279,332","$260,812"
Cost of Goods Sold,"$234,856","$163,605","$132,759","$159,053","$190,752","$181,228"
Gross Profit,"$129,907","$77,801","$67,869","$78,109","$88,580","$79,584"
R&D Expense,-,-,-,-,-,-
SG&A Expense,"$12,002","$10,961","$11,783","$11,893","$12,300","$12,094"
Depreciation & Amortization Expense,"$17,297","$18,048","$18,708","$17,893","$18,045","$18,403"
Other Operating Expenses,"$64,857","$32,834","$31,375","$32,459","$35,230","$33,161"
Operating Expense,"$94,156","$61,843","$61,866","$62,245","$65,575","$63,658"
Earnings Before Interest and Taxes,"$34,082","$14,435","$4,536","$14,074","$21,539","$14,459"
Interest Expense,$286,$311,$453,$601,$766,$844


Variable access becomes even easier, because now you have access to everything from both the income statement and balance sheet, without worrying about where it came from.

In [46]:
stmts.cash

2014-12-31    4616
2015-12-31    3705
2016-12-31    3657
2017-12-31    3177
2018-12-31    3042
2019-09-30    5351
dtype: int64

In [47]:
stmts.cogs

2014-12-31    234856
2015-12-31    163605
2016-12-31    132759
2017-12-31    159053
2018-12-31    190752
2019-09-30    181228
dtype: int64

We can also still access individual dates.

In [48]:
stmts['2014-12-31']

,12/31/2014
Revenue,"$364,763"
Cost of Goods Sold,"$234,856"
Gross Profit,"$129,907"
R&D Expense,-
SG&A Expense,"$12,002"
Depreciation & Amortization Expense,"$17,297"
Other Operating Expenses,"$64,857"
Operating Expense,"$94,156"
Earnings Before Interest and Taxes,"$34,082"
Interest Expense,$286


In [49]:
stmts[['2014-12-31', '2015-12-31']]

,12/31/2014,12/31/2015
Revenue,"$364,763","$241,406"
Cost of Goods Sold,"$234,856","$163,605"
Gross Profit,"$129,907","$77,801"
R&D Expense,-,-
SG&A Expense,"$12,002","$10,961"
Depreciation & Amortization Expense,-,-
Other Operating Expenses,"$64,857","$32,834"
Operating Expense,"$94,156","$61,843"
Earnings Before Interest and Taxes,"$34,082","$14,435"
Interest Expense,$286,$311


In [50]:
stmts[['2014-12-31', '2015-12-31']].total_equity

2014-12-31    181064.0
2015-12-31    176810.0
dtype: float64

### Calculating FCFs Using `finstmt`

I have already built FCF calculation into the package. So we can simply do:

In [51]:
stmts.fcf

2014-12-31        NaN
2015-12-31    15916.0
2016-12-31    18605.0
2017-12-31    10385.0
2018-12-31    28094.0
2019-09-30    21350.0
dtype: float64

If we wanted to calculate it manually for some reason, it would also be much easier. This not only because all the items are consolidated, but also calculating lags and changes is easier.

In [52]:
stmts.net_ppe

2014-12-31    252668
2015-12-31    251605
2016-12-31    244224
2017-12-31    252630
2018-12-31    247101
2019-09-30    257065
dtype: int64

In [53]:
stmts.lag('net_ppe', 1)

2014-12-31         NaN
2015-12-31    252668.0
2016-12-31    251605.0
2017-12-31    244224.0
2018-12-31    252630.0
2019-09-30    247101.0
dtype: float64

In [54]:
stmts.change('net_ppe')

2014-12-31       NaN
2015-12-31   -1063.0
2016-12-31   -7381.0
2017-12-31    8406.0
2018-12-31   -5529.0
2019-09-30    9964.0
dtype: float64

Also the components of FCF are precalculated.

In [55]:
stmts.net_income + stmts.non_cash_expenses - stmts.change('nwc') - stmts.capex

2014-12-31        NaN
2015-12-31    15916.0
2016-12-31    18605.0
2017-12-31    10385.0
2018-12-31    28094.0
2019-09-30    21350.0
dtype: float64

### Other Notes

`finstmt` is a general package which can work with a variety of data providers' financial statements. So far, I have confirmed it working with statements from Stockrow and from Capital IQ. Now you can write one set of code that will work regardless of where you are getting your data.

There may still be some rough edges. I put this package together in a couple days. It also has features for forecasting that we will cover in our next lecture. Please let me know if you run into any issues with the package.